In [1]:
import json
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
%matplotlib inline

## Workflow

In [50]:
df = pd.read_excel('MCC_Groups.xlsx', index=False)
temp = pd.DataFrame()

In [51]:
temp['mcc_description'] = df.mcc_description

In [52]:
temp['mcc_segment_description'] = df.mcc_segment_description

In [61]:
df.columns = ['merch_id', 'merch_name', 'cat_id', 'cat_name']

In [67]:
df.T.to_json(path_or_buf='cat_lookup.js')

In [87]:
import re

s = df.T.to_json()
re.sub(pattern='(["][0-9]+["][:])', string=s, repl="")

'{{"merch_id":3015,"merch_name":"SWISSAIR","cat_id":1,"cat_name":"Airlines"},{"merch_id":3021,"merch_name":"AIR ALGERIE","cat_id":1,"cat_name":"Airlines"},{"merch_id":3231,"merch_name":"SASHA SERVICIO AERO DE HONDURAS","cat_id":1,"cat_name":"Airlines"},{"merch_id":3513,"merch_name":"WESTIN HOTELS","cat_id":13,"cat_name":"Hotels and Motels"},{"merch_id":3528,"merch_name":"RED LION INNS","cat_id":13,"cat_name":"Hotels and Motels"},{"merch_id":3536,"merch_name":"AMFAC HOTELS","cat_id":13,"cat_name":"Hotels and Motels"},{"merch_id":3558,"merch_name":"JOLLY HOTELS","cat_id":13,"cat_name":"Hotels and Motels"},{"merch_id":3620,"merch_name":"TELFORD INTERNATIONAL","cat_id":13,"cat_name":"Hotels and Motels"},{"merch_id":3646,"merch_name":"SWALLOW HOTELS","cat_id":13,"cat_name":"Hotels and Motels"},{"merch_id":3675,"merch_name":"INTERHOTEL CEDOK","cat_id":13,"cat_name":"Hotels and Motels"},{"merch_id":3682,"merch_name":"ALLSTAR INNS","cat_id":13,"cat_name":"Hotels and Motels"},{"merch_id":3736,"

In [94]:
df.cat_name.unique()

array(['Airlines', 'Hotels and Motels', 'Transportation',
       'Professional Services', 'Business Services',
       'Miscellaneous Stores', 'Automobiles and Vehicles',
       'Clothing Stores', 'Restaurants', 'High Risk Personal Retail',
       'Mail Phone Order', 'Service Providers', 'Personal Services',
       'High Risk Personal Services', 'Repair Services',
       'Amusement and Entertainment', 'Education', 'Government Services',
       'Contracted Services', 'Wholesale Trade', 'Utilities',
       'Telecom and Data Utilities', 'Retail Stores', 'Associations',
       'Gas Stations', 'Publishing Services', 'Cleaning Preparations',
       'Auto Rental'], dtype=object)

In [88]:
# Replace "localhost:8000" with your PANDA's DNS name or IP address
PANDA_API = 'https://3hkaob4gkc.execute-api.us-east-1.amazonaws.com/prod/au-hackathon/transactions/'

# # Replace these parameters with your administrator's email and api key
# PANDA_AUTH_PARAMS = {
#     'email': 'panda@pandaproject.net',
#     'api_key': 'edfe6c5ffd1be4d3bf22f69188ac6bc0fc04c84b'
# }

# # # This will be the slug of the dataset we are creating/updating
# # PANDA_DATASET_SLUG = 'twitter-pandaproject'

# # This is the url where the dataset will live, we send our GET/PUT requests here
# PANDA_DATASET_URL = '%s/dataset/%s/' % (PANDA_API, PANDA_DATASET_SLUG)

# # This is the url under which all data for this dataset lives
# PANDA_DATA_URL = '%s/dataset/%s/data/' % (PANDA_API, PANDA_DATASET_SLUG)

# # Change this value to configure how many rows should be sent to PANDA in each batch
# PANDA_BULK_UPDATE_SIZE = 1000

# # This is the url of the Scraperwiki endpoint for our Twitter scraper
# # This was generated from: https://scraperwiki.com/docs/api#sqlite
# SCRAPERWIKI_URL = 'https://api.scraperwiki.com/api/1.0/datastore/sqlite?format=jsonlist&name=basic_twitter_scraper_437&query=select%20*%20from%20%60swdata%60'

# # These are the three columns in our dataset
# COLUMNS = ['text', 'id', 'from_user']



In [89]:
# Wrapper around a GET request
def panda_get(url, params={}):
    params.update(PANDA_AUTH_PARAMS)
    return requests.get(url, params=params)

# Wrapper around a PUT request
def panda_put(url, data, params={}):
    params.update(PANDA_AUTH_PARAMS)
    return requests.put(url, data, params=params, headers={ 'Content-Type': 'application/json' })


In [337]:
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer, TfidfTransformer


def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

# Let's create some stop words. I chose these values after doing a little bit of EDA.
stop =[]
#stop = stop + ['https', 'www', 'com', 'http']
cvt = TfidfVectorizer(stop_words=stop, analyzer='char', ngram_range=[2,8], norm='l1', lowercase=True)

# # Here we are initializing the values we want to grid search over.
# param_grid = dict(vect = [CountVectorizer(max_features=200000)],
#                   vect__ngram_range=[[1,4]], # Trying different ngram ranges
#                   vect__stop_words = [stop],
#                 tfidf = [TfidfTransformer()],
#                 tfidf__norm = [None],
#                 clf=[LogisticRegression()],
#                 clf__C=[.06, .07], # Trying different coefficients for alpha
#                 clf__penalty=['l1'])
            

# pipeline = Pipeline([
#     ('vect', cvt),
#     ('tfidf', TfidfTransformer(norm=None)),
#     ('clf', LogisticRegression(penalty='l1'))
# ]) 

# grid_search = GridSearchCV(pipeline, param_grid=param_grid)

# grid_search.fit(X_train.text, y_train)

In [338]:
model = cvt.fit_transform(y=list(df.cat_name) + list(df.cat_name.unique()), raw_documents=list(df.merch_name.str.replace("'","")) + list(df.cat_name.unique()))

In [339]:
temp = cvt.transform(raw_documents=pd.read_csv('categories.txt').T.index.str.replace('"',""))

In [340]:
data = pd.DataFrame(model.todense(), columns=cvt.get_feature_names())
data1 =pd.DataFrame(temp.todense(), columns=cvt.get_feature_names())

In [341]:
lr = LogisticRegression(solver='newton-cg', multi_class = 'multinomial').fit(X=data, y=list(df.cat_name) + list(df.cat_name.unique()))

In [342]:
list(zip(list(pd.read_csv('categories.txt')),lr.predict(X=data1), lr.predict_proba(X=data1)))

[('#03122 jewel',
  'Hotels and Motels',
  array([ 0.28776991,  0.018925  ,  0.00601937,  0.00171916,  0.01203902,
          0.0189231 ,  0.00171952,  0.00773932,  0.01031972,  0.00601989,
          0.00257936,  0.00515712,  0.00974434,  0.00601842,  0.35323681,
          0.00860115,  0.04043759,  0.01031896,  0.01978558,  0.00257929,
          0.01210608,  0.00429832,  0.01547261,  0.00515894,  0.00429975,
          0.10050853,  0.00429953,  0.02420362])),
 (' "abc*anytime fitness"',
  'Hotels and Motels',
  array([ 0.30121093,  0.01909832,  0.00603476,  0.00172111,  0.01207158,
          0.01901274,  0.00172181,  0.00775589,  0.01032928,  0.00603356,
          0.00258303,  0.00520404,  0.00947159,  0.00602739,  0.33932875,
          0.00861334,  0.04062044,  0.01033291,  0.01987558,  0.00258241,
          0.01204911,  0.0043049 ,  0.01552114,  0.00516937,  0.00430611,
          0.10052583,  0.0043106 ,  0.02418349])),
 (' "abc*club fitness"',
  'Hotels and Motels',
  array([ 0.296795

In [343]:
merchants = pd.DataFrame(
    {'merchant': list(pd.read_csv('categories.txt')),
     'category': list(lr.predict(X=data1)),
     'prob': lr.predict_proba(X=data1)[:,0]
    })


In [460]:
for i in range(len(merchants)):
           
    if merchants.get_value(col='prob', index=i) < .29:
         merchants.set_value(col='category', index=i, value='Other')
    
    if 'parking' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Transportation')
    if 'air' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Airlines')
    if 'flight' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Airlines')
    if 'international' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Airlines')
    if 'com' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Miscellaneous Stores')
    if 'google' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Amusement and Entertainment')
    if 'play' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Amusement and Entertainment')
    if 'game' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Amusement and Entertainment')
    if 'cinema' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Amusement and Entertainment')
    if 'theat' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Amusement and Entertainment')
    if 'school' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Education')
    if 'gas' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Gas Stations')
    if 'fuel' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Gas Stations')
    if 'car' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Automobiles and Vehicles')
    if 'wireless' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Telecom and Data Utilities')
    if 'plan' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Telecom and Data Utilities')
    if 'data' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Telecom and Data Utilities')
    if 'mobile' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Telecom and Data Utilities')
    if 'care' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Personal Services')
    if 'fitness' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Personal Services')
    if 'dental' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Personal Services')
    if 'sport' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Personal Services')
    if 'weight' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Personal Services')
    if 'ymca' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Personal Services')
    if 'health' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Personal Services')
    if 'gym' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Personal Services')
    if 'airbnb' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'costco' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Retail Stores')
    if 'amazon' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Retail Stores')
    if 'mart' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Retail Stores')
    if 'gov' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Government Services')
    if 'dmv' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Government Services')
    if 'state' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Government Services')
    if 'pub' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Government Services')
    if 'county' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Public Services')
    if 'expres' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Miscellaneous Stores')
    if 'hotel' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'hya' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'hamil' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'motel' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'resort' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'casino' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'sui' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'inn' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'tropic' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'beach' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'club' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'trav' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Hotels and Motels')
    if 'cigna' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Personal Services')
    if 'inc' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Professional Services')
    if 'llc' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Professional Services')
    if 'paypal' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Professional Services')
    if 'auto' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Repair Services')
    if 'kais' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Professional Services')
    if 'insu' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Professional Services')
    if 'photo' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Professional Services')
    if 'macy' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Clothing Store')
    if 'crew' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Clothing Store')
    if 'kohl' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Clothing Store')
    if 'nord' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Clothing Store')
    if 'penney' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Clothing Store')
    if 'coral' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Restaurants')
    if 'food' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Restaurants')
    if 'cheese' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Restaurants')
    if 'corral' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Restaurants')
    if 'apron' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Restaurants')
    if 'kreme' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Restaurants')
    if 'buster' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Restaurants')
    if 'farm' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Restaurants')
    if 'steak' in merchants.get_value(col='merchant', index=i).lower():
        merchants.set_value(col='category', index=i, value='Restaurants')

In [461]:
merchants.category.value_counts()

Other                          58
Hotels and Motels              51
Professional Services          36
Miscellaneous Stores           27
Personal Services              26
Amusement and Entertainment    20
Retail Stores                  11
Restaurants                    10
Telecom and Data Utilities      9
Transportation                  8
Airlines                        7
Government Services             6
Automobiles and Vehicles        6
Clothing Store                  5
Public Services                 5
Education                       3
Gas Stations                    1
Repair Services                 1
Name: category, dtype: int64

In [462]:
merchants[merchants.category == 'Other'].sort_values(by='prob')

,category,merchant,prob
243,Other,"""stubhub",0.285385
172,Other,"""msb huntley consolidated""",0.285580
251,Other,"""the rose bowl""",0.285936
96,Other,"""fidelity national home""",0.286102
70,Other,"""corelogic""",0.286381
140,Other,"""intuit *quickbooks se""",0.287022
215,Other,"""ramada - ram 08255""",0.287338
137,Other,"""identity guard""",0.287589
0,Other,#03122 jewel,0.287770
181,Other,"""office depot #161""",0.288515


In [287]:
lr.classes_

array(['Airlines', 'Amusement and Entertainment', 'Associations',
       'Auto Rental', 'Automobiles and Vehicles', 'Business Services',
       'Cleaning Preparations', 'Clothing Stores', 'Contracted Services',
       'Education', 'Gas Stations', 'Government Services',
       'High Risk Personal Retail', 'High Risk Personal Services',
       'Hotels and Motels', 'Mail Phone Order', 'Miscellaneous Stores',
       'Personal Services', 'Professional Services', 'Publishing Services',
       'Repair Services', 'Restaurants', 'Retail Stores',
       'Service Providers', 'Telecom and Data Utilities', 'Transportation',
       'Utilities', 'Wholesale Trade'], dtype=object)

In [472]:
merchants.columns = ['cat_name', 'merch_name']

In [478]:
import re
merchants['merch_name'] = merchants.merch_name.str.replace('"',"")
merchants['merch_name'] = merchants.merch_name.str.strip()
s = merchants.T.to_json()
re.sub(pattern='(["][0-9]+["][:])', string=s, repl="")



'{{"cat_name":"Other","merch_name":"#03122 jewel"},{"cat_name":"Personal Services","merch_name":"abc*anytime fitness"},{"cat_name":"Hotels and Motels","merch_name":"abc*club fitness"},{"cat_name":"Personal Services","merch_name":"abc*golds gym"},{"cat_name":"Personal Services","merch_name":"abc*planet fitness"},{"cat_name":"Personal Services","merch_name":"abc*retro fitness"},{"cat_name":"Personal Services","merch_name":"act ymca"},{"cat_name":"Hotels and Motels","merch_name":"advanced disposal"},{"cat_name":"Personal Services","merch_name":"aetna health managemen"},{"cat_name":"Personal Services","merch_name":"aetna medicare"},{"cat_name":"Personal Services","merch_name":"affinity health plan"},{"cat_name":"Professional Services","merch_name":"affinity sports llc"},{"cat_name":"Transportation","merch_name":"ahec parking services"},{"cat_name":"Hotels and Motels","merch_name":"airbnb"},{"cat_name":"Airlines","merch_name":"airport parking"},{"cat_name":"Airlines","merch_name":"alaska ai

In [477]:
merchants

,cat_name,merch_name
0,Other,#03122 jewel
1,Personal Services,abc*anytime fitness
2,Hotels and Motels,abc*club fitness
3,Personal Services,abc*golds gym
4,Personal Services,abc*planet fitness
5,Personal Services,abc*retro fitness
6,Personal Services,act ymca
7,Hotels and Motels,advanced disposal
8,Personal Services,aetna health managemen
9,Personal Services,aetna medicare
